In [1]:
from io import StringIO

import os
import re
import PyPDF2
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure, LAParams
from ftlangdetect import detect

In [2]:
def collect_clear_text(text_list):
    text_list = [re.sub(r'\s+', ' ', text_.strip()).strip() for text_ in text_list]
    text = ' '.join(text_list)
    return text.strip()

def contents_references_rmv(text_per_page: dict,pages = 30, char_num = 600, years_num = 10):
    
    def less_char_drop(text,char_num):
        if len(text) < char_num: return True
    def dot_drop(text):
        if (len(text) > char_num) and ((text.count(".") > 40) or (text.count("…")> 10) or (text.count("_")> 10) or (text.count("-")>10)) : return True
    def referencesYear_drop(text):
        year_list = list(range(1900,2025))
        year_count = 0
        for year in year_list:
            if str(year) in text:
                year_count += 1
        if year_count > years_num: return True
    total_pages = len(text_per_page)
    obs_numbers = list(range(0,pages)) + list(range(total_pages-pages,total_pages))
    for page_num in obs_numbers:
        page_word = f'Page_{page_num}'
        if page_num < pages:
            try:
                if (less_char_drop(text_per_page[page_word],char_num)) or (dot_drop(text_per_page[page_word]))  : del text_per_page[page_word]   
            except KeyError:
                continue
        else:
            if (less_char_drop(text_per_page[page_word],char_num)) or (referencesYear_drop(text_per_page[page_word]))  : del text_per_page[page_word]
    return text_per_page

def remove_eng(text_per_page: dict):
    dict_keys = list(text_per_page.keys())
    for keys in dict_keys: 
        text = text_per_page[keys]
        lang = detect(text)
        #print(keys, lang)
        if (lang["lang"] == 'tr') and (lang["score"] < 0.75):
            del text_per_page[keys]
        elif lang["lang"] != "tr":
            del text_per_page[keys]
    return text_per_page

def remove_pageNum(text: str):
    text = text.strip(" ")
    text_list = text.split(" ")
    if text_list[0].strip().isnumeric():
        text_list.remove(text_list[0])
    if text_list[-1].strip().isnumeric():
        text_list.remove(text_list[-1])

    return " ".join(text_list).strip()


In [3]:
pdf_listDir = os.listdir("istanbul_baro_pdf") 
for pdf in pdf_listDir:
    text_per_page = {}
    pdf_path = f'istanbul_baro_pdf/{pdf}'
    for pagenum, page in enumerate(extract_pages(pdf_path, laparams = LAParams(line_margin = 2))):
        page_text = []
        page_elements = [(element.y1, element) for element in page._objs]
        page_elements.sort(key=lambda a: a[0], reverse=True)
        for i, component in enumerate(page_elements):
            element = component[1]
            if isinstance(element, LTTextContainer):
                line_text = element.get_text()
                page_text.append(line_text)
        dctkey = 'Page_'+str(pagenum)
        text_per_page[dctkey]= collect_clear_text(page_text)
    rawTotal = len(text_per_page)
    text_per_page = contents_references_rmv(text_per_page=text_per_page)
    text_per_page = remove_eng(text_per_page=text_per_page)

    clearedTotal = len(text_per_page)
    if clearedTotal < 2:
        print("Bozuk pdf:", pdf)
    print("PDF:", pdf, "Raw_Pages:", rawTotal, "Cleared_Total:", clearedTotal)
    with open(f"txt_folder/{pdf[:-4]}.txt" , 'w+', encoding="utf-8") as txt:
        for text in text_per_page.values():
            txt.write(remove_pageNum(text))
            txt.write('\n')
    os.rename( f"istanbul_baro_pdf/{pdf}", f"istBaroPDF_fin/finished_{pdf}")

PDF: 10.pdf Raw_Pages: 561 Cleared_Total: 497
PDF: 11.pdf Raw_Pages: 673 Cleared_Total: 609
PDF: 12.pdf Raw_Pages: 665 Cleared_Total: 600
PDF: 14.pdf Raw_Pages: 767 Cleared_Total: 705
PDF: 15.pdf Raw_Pages: 567 Cleared_Total: 502
PDF: 16.pdf Raw_Pages: 641 Cleared_Total: 558
PDF: 17.pdf Raw_Pages: 805 Cleared_Total: 729
PDF: 18.pdf Raw_Pages: 849 Cleared_Total: 744
PDF: 19.pdf Raw_Pages: 713 Cleared_Total: 638
PDF: 200705ibshos.pdf Raw_Pages: 317 Cleared_Total: 277
PDF: 3.pdf Raw_Pages: 414 Cleared_Total: 358
PDF: 4.pdf Raw_Pages: 393 Cleared_Total: 351
PDF: 5.pdf Raw_Pages: 529 Cleared_Total: 480
PDF: 6.pdf Raw_Pages: 545 Cleared_Total: 490
PDF: 7.pdf Raw_Pages: 593 Cleared_Total: 529
PDF: 8.pdf Raw_Pages: 577 Cleared_Total: 504
Bozuk pdf: ahos2007.pdf
PDF: ahos2007.pdf Raw_Pages: 2 Cleared_Total: 0
PDF: cos2006.pdf Raw_Pages: 325 Cleared_Total: 251
PDF: ibd20061.pdf Raw_Pages: 464 Cleared_Total: 361
PDF: ibd20062.pdf Raw_Pages: 497 Cleared_Total: 397
PDF: ibd20063.pdf Raw_Pages: 522 